In [1]:
import json
import os
from collections import defaultdict
import pdfplumber  # 用於從PDF文件中提取文字的工具
import os
import json
import argparse
import numpy as np
from tqdm import tqdm
import jieba  # 用於中文文本分詞
import pdfplumber  # 用於從PDF文件中提取文字的工具
from rank_bm25 import BM25Okapi  # 使用BM25演算法進行文件檢索
from collections import defaultdict
import torch
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel

/home/tony/anaconda3/envs/NLP/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# 讀取單個PDF文件並返回其文本內容
def read_pdf(pdf_loc, page_infos: list = None):
    pdf = pdfplumber.open(pdf_loc)  # 打開指定的PDF文件

    # TODO: 可自行用其他方法讀入資料，或是對pdf中多模態資料（表格,圖片等）進行處理

    # 如果指定了頁面範圍，則只提取該範圍的頁面，否則提取所有頁面
    pages = pdf.pages[page_infos[0]:page_infos[1]] if page_infos else pdf.pages
    pdf_text = ''
    for _, page in enumerate(pages):  # 迴圈遍歷每一頁
        text = page.extract_text()  # 提取頁面的文本內容
        if text:
            pdf_text += text
    pdf.close()  # 關閉PDF文件

    return pdf_text  # 返回萃取出的文本

In [4]:
import os
from tqdm import tqdm

def load_category_corpus(source_path, source_ids):
    """
    只讀取指定 source_ids 中的文件，以減少不必要的讀取。
    :param source_path: 資料夾路徑
    :param source_ids: 要讀取的文件 id 列表
    :return: 該類別對應的 corpus 字典
    """
    corpus_dict = {}
    for file in tqdm(os.listdir(source_path)):
        file_id = int(file.replace('.pdf', ''))
        if file_id in source_ids:  # 僅載入在 source_ids 中的文件
            file_path = os.path.join(source_path, file)
            corpus_dict[file_id] = read_pdf(file_path)
    return corpus_dict

def get_unique_source_ids(qs_ref, category):
    """
    根據指定的 category，獲取所有 unique source ids。
    :param qs_ref: 問題的 JSON 資料
    :param category: 類別名稱 (如 'insurance', 'finance')
    :return: 該類別下的 unique source ids 集合
    """
    source_ids = set()
    for question in qs_ref["questions"]:
        if question["category"] == category:
            source_ids.update(question["source"])
    return sorted(source_ids)

# 使用範例
with open('競賽資料集/dataset/preliminary/questions_example.json', 'rb') as f:
    qs_ref = json.load(f)

source_path_insurance = os.path.join('競賽資料集/reference', 'insurance')
insurance_source_ids = get_unique_source_ids(qs_ref, 'insurance')
corpus_dict_insurance = load_category_corpus(source_path_insurance, insurance_source_ids)

source_path_finance = os.path.join('競賽資料集/reference', 'finance')
finance_source_ids = get_unique_source_ids(qs_ref, 'finance')
corpus_dict_finance = load_category_corpus(source_path_finance, finance_source_ids)


100%|██████████| 1035/1035 [00:41<00:00, 24.82it/s]


In [3]:
model = SentenceTransformer('BAAI/bge-m3', device='cuda') # 使用BGE模型 m3 AP 0.77
x  = ['hello world', 'goodbye world']
embeddings = model.encode(x)
embeddings.shape

(2, 1024)

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=False)


def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]
def get_embeddings(text_list):
    model = SentenceTransformer('BAAI/bge-m3', device='cuda') # 使用BGE模型 m3 AP 0.77
    tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to('cuda') for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

corpus = [
    "BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction."
]
query = ["What is BGE M3?"]

pair = [[i,j] for i in query for j in corpus]
# passage_embeddings = model.encode(
#     corpus, return_dense=True, return_sparse=True, return_colbert_vecs=True
# )
print(model.compute_score(pair)['colbert'])

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 296766.79it/s]


[0.7796663641929626]


In [11]:
passage_embeddings['dense_vecs']

array([[-0.00011137, -0.06657315, -0.00018456, ...,  0.04317538,
        -0.02131795,  0.01383029]], dtype=float32)

In [ ]:


def Bge_retrieve(qs, source, corpus_dict, category):
    # 將文檔內容列出來以文字形式
    filtered_corpus = [corpus_dict[int(file)] for file in source]
    #將文字內容轉為向量 embeddings
    model = SentenceTransformer('BAAI/bge-m3', device='cuda') # 使用BGE模型 m3 AP 0.77
    corpus_embeddings = model.encode(filtered_corpus, prompt=f'以下是關於{category}的文章', convert_to_tensor=True, device='cuda' )
    query_embedding = model.encode(qs,prompt="為這個句子生成表示，用以檢索相似的文章: " ,convert_to_tensor=True, device='cuda')

    # 計算相似度
    similarities = torch.nn.functional.cosine_similarity(query_embedding, corpus_embeddings)
    max_sim_index = similarities.argmax().item()
    
    return source[max_sim_index]

answer_dict = {"answers": []}  # 初始化字典
with open(os.path.join("競賽資料集/reference", 'faq/pid_map_content.json'), 'rb') as f_s:
    key_to_source_dict = json.load(f_s)  # 讀取參考資料文件
    key_to_source_dict = {int(key): value for key, value in key_to_source_dict.items()}

for q_dict in qs_ref['questions']:
    if q_dict['category'] == 'finance':
        # 進行檢索
        retrieved = Bge_retrieve(q_dict['query'], q_dict['source'], corpus_dict_finance, "公開資訊觀測站上的上市公司財務報告")
        # 將結果加入字典
        answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved})

    elif q_dict['category'] == 'insurance':
        retrieved = Bge_retrieve(q_dict['query'], q_dict['source'], corpus_dict_insurance, "⽟⼭銀⾏代銷的保險產品之保單條款" )
        answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved})

    elif q_dict['category'] == 'faq':
        corpus_dict_faq = {key: str(value) for key, value in key_to_source_dict.items() if key in q_dict['source']}
        retrieved = Bge_retrieve(q_dict['query'], q_dict['source'], corpus_dict_faq, "⽟⼭銀⾏官⽅網站上的常⾒問題")
        answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved})

    else:
        raise ValueError("Something went wrong")  # 如果過程有問題，拋出錯誤

# 將答案字典保存為json文件
with open('result/bge_add_cat.json', 'w', encoding='utf8') as f:
    json.dump(answer_dict, f, ensure_ascii=False, indent=4)  # 儲存檔案，確保格式和非ASCII字符


/home/tony/anaconda3/envs/NLP/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [16]:
source = ['1001', '1002', '1003', '1004']
corpus = ['this is a dog', 'this is a cat', 'this is a puppy', 'this is a kitten']
model = SentenceTransformer('BAAI/bge-m3', device='cuda')
query = "這是一隻狗的英文怎麼說"
corpus_embeddings = model.encode(corpus,convert_to_tensor=True, device='cuda')
query_embedding = model.encode(query,convert_to_tensor=True, device='cuda')
similarities = torch.nn.functional.cosine_similarity(query_embedding, corpus_embeddings)
print(similarities)

tensor([0.8213, 0.7396, 0.7956, 0.7409], device='cuda:0')


In [4]:
import json

with open('競賽資料集/dataset/preliminary/questions_example.json', 'r') as f:
    data = json.load(f)


# 找出最小的 source 陣列長度
min_source_count = min(len(question["source"]) for question in data["questions"])

print(f"Source 最少的數量為: {min_source_count}")


Source 最少的數量為: 2


In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
# Sentences we want sentence embeddings for
sentences = ["样例数据-1", "样例数据-2"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5')
model.eval()

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
print("Sentence embeddings:", sentence_embeddings)


/home/tony/anaconda3/envs/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings: tensor([[ 0.0015,  0.0165, -0.0281,  ..., -0.0309,  0.0297, -0.0327],
        [ 0.0151,  0.0041, -0.0157,  ..., -0.0281,  0.0408, -0.0251]])


In [2]:
sentence_embeddings.shape

torch.Size([2, 1024])

In [10]:
x = sentence_embeddings[0] @ sentence_embeddings[1].T
x.tolist()

0.8791519999504089